In [1]:
import pandas as pd
import numpy as np

In [6]:
#importation des données
credit2=pd.ExcelFile("credit2.xlsx")
frameCredit2=credit2.parse()
frameCredit1=pd.read_table("credit1.txt")
frameCreditGlobal=pd.merge(frameCredit1,frameCredit2,how='left',on="Customer_ID")

In [50]:
#création de x et y pour le modèle
y=frameCreditGlobal["Accepted_Credit_Card"]
col_pred=["Income","Age","RFM_Score","Average_Balance_Feed_Index"]
x=frameCreditGlobal[col_pred]

In [51]:
from sklearn.cross_validation import train_test_split

In [52]:
#création des échantillons d'apprentissage et de test
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.3)

In [53]:
from sklearn.neighbors import KNeighborsClassifier

In [54]:
#création de l'objet knn avec 20 voisins
knn=KNeighborsClassifier(n_neighbors=20)


In [55]:
#ajustement du modèle sur les données d'apprentissage
knn.fit(x_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=20, p=2,
           weights='uniform')

In [56]:
#prédiction avec les données test
prediction=knn.predict(x_test)

In [57]:
#calcul de l'accuracy
accuracy=(prediction==y_test).sum()/float(len(prediction))

In [59]:
from sklearn.metrics import confusion_matrix, accuracy_score

In [60]:
#calcul de l'accuracy
accuracy_score(y_test,prediction)

0.89166666666666672

In [61]:
#calcul de la matrice de confusion
confusion_matrix(y_test,prediction)

array([[535,   0],
       [ 65,   0]])

In [63]:
#utilisation des SVM
from sklearn.svm import SVC 

In [64]:
#création de l'objet
modelSVM=SVC()

In [65]:
#ajustement du modèle
modelSVM.fit(x_train,y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [66]:
#prédiction avec le modèle et les données de validation
predSVM=modelSVM.predict(x_test)

In [69]:
confusion_matrix(y_test,predSVM)

array([[533,   2],
       [ 64,   1]])

In [70]:
accuracy_score(y_test, predSVM)

0.89000000000000001

In [71]:
#grid search pour SVM
from sklearn.grid_search import GridSearchCV

In [72]:
#définition des hyper paramètres
hyperparam=[{'kernel':['rbf'], 'gamma':[.001,.0001], 'C':[1,10]},
            {'kernel':['linear'], 'C':[1,10]}]
hyperparam

[{'C': [1, 10], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']},
 {'C': [1, 10], 'kernel': ['linear']}]

In [74]:
#création de l'objet de recherche d'hyper-paramètres
gridModelSVM=GridSearchCV(SVC(),hyperparam,scoring='accuracy',cv=5)

In [75]:
#ajustemement de 30 modèles SVM
gridModelSVM.fit(x_train,y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid=[{'kernel': ['rbf'], 'C': [1, 10], 'gamma': [0.001, 0.0001]}, {'kernel': ['linear'], 'C': [1, 10]}],
       pre_dispatch='2*n_jobs', refit=True, scoring='accuracy', verbose=0)

In [77]:
#affichage des paramètres les meilleurs
gridModelSVM.best_params_

{'C': 1, 'kernel': 'linear'}

In [79]:
#affichage des résultats de valid croisée
gridModelSVM.grid_scores_

[mean: 0.87286, std: 0.00668, params: {'kernel': 'rbf', 'C': 1, 'gamma': 0.001},
 mean: 0.87929, std: 0.00282, params: {'kernel': 'rbf', 'C': 1, 'gamma': 0.0001},
 mean: 0.86357, std: 0.01293, params: {'kernel': 'rbf', 'C': 10, 'gamma': 0.001},
 mean: 0.84786, std: 0.01359, params: {'kernel': 'rbf', 'C': 10, 'gamma': 0.0001},
 mean: 0.88357, std: 0.00142, params: {'kernel': 'linear', 'C': 1},
 mean: 0.88357, std: 0.00142, params: {'kernel': 'linear', 'C': 10}]